### Extracting Key words using rake and scoring them using word page

In [ ]:


P=0
#Question arrays
str_arr=["After reading the group’s procedure, describe what additional information you would need in order to replicate the experiment. Make sure to include at least three pieces of information.",
        "Draw a conclusion based on the student’s data. Describe two ways the student could have improved the experimental design and/or validity of the results.",
        "Explain how pandas in China are similar to koalas in Australia and how they both are different from pythons. Support your response with information from the article.",
        "Explain the significance of the word “invasive” to the rest of the article. Support your response with information from the article.",
        "Starting with mRNA leaving the nucleus, list and describe four major steps involved in protein synthesis.",
        "List and describe three processes used by cells to control the movement of substances across the cell membrane.",
        "Identify ONE trait that can describe Rose based on her conversations with Anna or Aunt Kolab. Include ONE detail from the story that supports your answer.",
         "During the story, the reader gets background information about Mr. Leonard. Explain the effect that background information has on Paul. Support your response with details from the story.",
        "How does the author organize the article? Support your response with details from the article.",
        "Brandi and Jerry were designing a doghouse. Use the results from the experiment to describe the best paint color for the doghouse."]

#Calling Rake 
def extract_rake(str_arr,P):  

    
    for i in str_arr:
        P+=1
        from rake_nltk import Rake
        r = Rake()
        string = i
        r.extract_keywords_from_text(string)
        terms = []
        for x in r.get_ranked_phrases_with_scores():
            terms.append(x[1])

#using page rank to rank features (paper corrosponding link)
        try: 
            from googlesearch import search 
        except ImportError: 
            print("No module named 'google' found") 

        urls = []
        for query in terms:
            for j in search(query,lang='en', num=1, stop=5, pause=2):
                urls.append(j)
   

        import nltk, re, pprint
        import numpy as np
        from nltk import word_tokenize

        docs = []

        import urllib.parse
        import urllib.request
        from urllib.error import HTTPError
        # import requests 
        from bs4 import BeautifulSoup

        for url in urls:
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
            reg_url = url
            req = urllib.request.Request(url=reg_url, headers=headers) 
        
            try:
                html = urllib.request.urlopen(req).read()
                soup = BeautifulSoup(html)

                
                for script in soup(["script", "style"]):
                    script.extract()    # rip it out

                # get text
                text = soup.get_text()

                # break into lines and remove leading and trailing space on each
                lines = (line.strip() for line in text.splitlines())
                # break multi-headlines into a line each
                chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
                # drop blank lines
                text = '\n'.join(chunk for chunk in chunks if chunk)
                text = text.lower()
                symbols = "!\"#$%&()*,+-./:;<=>?@[\]^_`{|}~\n"
                for i in symbols:
                    text = text.replace(i, ' ')
                # print(text)
                docs.append(text)
            except HTTPError as e:
                content = e.read()

        print(len(docs))

        #import the TfidfVectorizer from Scikit-Learn.  
        from sklearn.feature_extraction.text import TfidfVectorizer
        vectorizer = TfidfVectorizer(stop_words=None, use_idf=True, norm=None)
        transformed_documents = vectorizer.fit_transform(docs)

        transformed_documents_as_array = transformed_documents.toarray()
        # use this line of code to verify that the numpy array represents the same number of documents that we have in the file list
        print(len(transformed_documents_as_array[1]))

        print(len(vectorizer.get_feature_names()))

        # print(len(transformed_documents_as_array[0]))

        import pandas as pd
        for counter, doc in enumerate(transformed_documents_as_array):
            # construct a dataframe
            tf_idf_tuples = list(zip(vectorizer.get_feature_names(), doc))
            one_doc_as_df = pd.DataFrame.from_records(tf_idf_tuples, columns=['term', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)
            one_doc_as_df.to_csv('weighted_keywords/set_' + str(P) +'.csv',index=None)


In [ ]:
extract_rake(str_arr,0)

### Loading Doc2Vec Model

In [ ]:

model = Doc2Vec.load('../data/embeddings/enwiki_dbow/doc2vec.bin')

### Using wikipedia article for the top word to calucatr cosine similarity between response and top word article

In [ ]:
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import pandas as pd
import wikipedia
#load pretrained model


# Load Data
data = pd.read_table('../data/train_rel_2.tsv')
print('read_data')
import numpy as np
from scipy import spatial
score_matrix = np.zeros(shape=(len(data),10))
count = -1
length = 0
sample_res_overlap = []

#Load weighted keywords for each prompt (saved by cell above)
df1= pd.read_csv('weighted_keywords/set_1.csv')
df2 = pd.read_csv('weighted_keywords/set_2.csv')
df3= pd.read_csv('weighted_keywords/set_3.csv')
df4 = pd.read_csv('weighted_keywords/set_4.csv')
df5= pd.read_csv('weighted_keywords/set_5.csv')
df6 = pd.read_csv('weighted_keywords/set_6.csv')
df7= pd.read_csv('weighted_keywords/set_7.csv')
df8 = pd.read_csv('weighted_keywords/set_8.csv')
df9= pd.read_csv('weighted_keywords/set_9.csv')
df10 = pd.read_csv('weighted_keywords/set_10.csv')

for row in data.iterrows():
    count = count + 1
    sent = row[1]["EssayText"]
    ans_set = row[1]["EssaySet"]
    sent = word_tokenize(sent)
    sent_vector = model.infer_vector(sent)
    flag=False



    if ans_set == 1:
#choose the top 1 word
        top_words = df1['term'].tolist()[:1]
        print(top_words)
 #calling wikipedia for article of the top word
        try : 
            article = wikipedia.page(top_words).content
        except:
#if top word cannot be retrieved using wikipedia pages, use the second top word
            article = wikipedia.page(df1['term'].tolist()[1:2]).content

        article = word_tokenize(article.lower())
        article_vec = model.infer_vector(article)
#find and append cosine similarity

        sample_res_overlap.append(spatial.distance.cosine(sent_vector, article_vec))

    if ans_set == 2:
        flag=True
        if(flag==True)
            sample_res_overlap1 = sample_res_overlap.copy()
            flag = False

        top_words = df2['term'].tolist()[:1]

        try : 
            article = wikipedia.page(top_words).content
        except:
            article = wikipedia.page(df2['term'].tolist()[1:2]).content
        article = word_tokenize(article.lower())
        article_vec = model.infer_vector(article)
       
        sample_res_overlap.append(spatial.distance.cosine(sent_vector, article_vec))

    if ans_set == 3:
        flag=True
        if(flag==True)
            sample_res_overlap2 = sample_res_overlap.copy()
            flag = False

        top_words = df3['term'].tolist()[:1]
        print(top_words)

        try : 
            article = wikipedia.page(top_words).content
        except :
            article = wikipedia.page(df3['term'].tolist()[1:2]).content
        article = word_tokenize(article.lower())
        article_vec = model.infer_vector(article)
  
        sample_res_overlap.append(spatial.distance.cosine(sent_vector, article_vec))

    if ans_set == 4:
        flag=True
        if(flag==True)
            sample_res_overlap3 = sample_res_overlap.copy()
            flag = False

        top_words = df4['term'].tolist()[:1]
        print(top_words)

        try : 
            article = wikipedia.page(top_words).content
        except :
            article = wikipedia.page(df4['term'].tolist()[1:2]).content
        article = word_tokenize(article.lower())
        article_vec = model.infer_vector(article)
       
        sample_res_overlap.append(spatial.distance.cosine(sent_vector, article_vec))

    if ans_set == 5:

        top_words = df5['term'].tolist()[:1]
        print(top_words)

        try : 
            article = wikipedia.page(top_words).content
        except:
            article = wikipedia.page(df5['term'].tolist()[1:2]).content
        article = word_tokenize(article.lower())
        article_vec = model.infer_vector(article)
   
        sample_res_overlap.append(spatial.distance.cosine(sent_vector, article_vec))
    if ans_set == 6:

        top_words = df6['term'].tolist()[:1]
        
        try : 
            article = wikipedia.page(top_words).content
        except:
            article = wikipedia.page(df6['term'].tolist()[1:2]).content
        article = word_tokenize(article.lower())
        article_vec = model.infer_vector(article)
     
        sample_res_overlap.append(spatial.distance.cosine(sent_vector, article_vec))

    if ans_set == 7:

        top_words = df7['term'].tolist()[:1]

        try : 
            article = wikipedia.page(top_words).content
        except:
            article = wikipedia.page(df7['term'].tolist()[1:2]).content
        article = word_tokenize(article.lower())
        article_vec = model.infer_vector(article)

        sample_res_overlap.append(spatial.distance.cosine(sent_vector, article_vec))

    if ans_set == 8:

        top_words = df8['term'].tolist()[:1]
        print(top_words)
 
        try : 
            article = wikipedia.page(top_words).content
        except:
            article = wikipedia.page(df8['term'].tolist()[1:2]).content
        article = word_tokenize(article.lower())
        article_vec = model.infer_vector(article)

        sample_res_overlap.append(spatial.distance.cosine(sent_vector,article_vec))

    if ans_set == 9:
        top_words = df9['term'].tolist()[:1]
        print(top_words)

        try : 
            article = wikipedia.page(top_words).content
        except:
            article = wikipedia.page(df9['term'].tolist()[1:2]).content
        article = word_tokenize(article.lower())
        article_vec = model.infer_vector(article)
    
        sample_res_overlap.append(spatial.distance.cosine(sent_vector, article_vec))
    if ans_set == 10:
        top_words = df10['term'].tolist()[:1]
        print(top_words)

        try: 
            article = wikipedia.page(top_words).content
        except:
            article = wikipedia.page(df10['term'].tolist()[1:2]).content
        article = word_tokenize(article.lower())
        article_vec = model.infer_vector(article)
        sample_res_overlap.append(spatial.distance.cosine(sent_vector, article_vec))

        #finallys saving feature
np.save("features/context_feature", sample_res_overlap)
